# RAG Chatbot Test Notebook

Bu notebook RAG (Retrieval Augmented Generation) chatbot sisteminin test edilmesi ve geliştirilmesi için kullanılır.

## İçerik:
1. Kütüphane kurulumu ve import'lar
2. Konfigürasyon ve ayarlar
3. Doküman yükleme ve işleme
4. RAG pipeline test'i
5. Performans değerlendirmesi
6. Örnek sorgu-cevap test'leri

## 1. Kütüphane Kurulumu ve Import'lar

In [ ]:
# Gerekli kütüphaneleri yükle
import sys
import os
from pathlib import Path

# Proje root dizinini sys.path'e ekle
project_root = Path().absolute()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[:3]}...")  # İlk 3 path'i göster

In [ ]:
# Kütüphane kontrolü ve import'lar
try:
    import langchain
    import chromadb
    import google.generativeai as genai
    from langchain.document_loaders import PyPDFLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.vectorstores import Chroma
    from langchain.embeddings import GoogleGenerativeAIEmbeddings
    from langchain.llms import GoogleGenerativeAI
    from langchain.chains import RetrievalQA
    
    print("✅ Tüm kütüphaneler başarıyla yüklendi")
    print(f"LangChain version: {langchain.__version__}")
    print(f"ChromaDB version: {chromadb.__version__}")
    
except ImportError as e:
    print(f"❌ Import hatası: {e}")
    print("Lütfen requirements.txt'teki kütüphaneleri yükleyin:")
    print("pip install -r requirements.txt")

In [ ]:
# Proje modüllerini import et
try:
    from src.rag_pipeline import RAGPipeline, create_rag_pipeline
    from src.document_processor import DocumentProcessor, validate_file
    from src.utils import setup_logging, check_api_key, Timer, truncate_text
    
    print("✅ Proje modülleri başarıyla yüklendi")
    
except ImportError as e:
    print(f"❌ Proje modülü import hatası: {e}")
    print("Lütfen src/ dizinindeki dosyaların mevcut olduğundan emin olun")

## 2. Konfigürasyon ve Ayarlar

In [ ]:
# Logging setup
setup_logging("INFO")
print("✅ Logging ayarlandı")

In [ ]:
# API Key konfigürasyonu
# Not: Bu hücreyi çalıştırmadan önce .env dosyanızda GEMINI_API_KEY'i ayarlayın
# veya aşağıdaki satırı uncomment edip API key'inizi girin

# os.environ["GEMINI_API_KEY"] = "your_api_key_here"

if check_api_key():
    print("✅ Gemini API Key ayarlandı")
else:
    print("❌ Gemini API Key bulunamadı")
    print("Lütfen .env dosyasında GEMINI_API_KEY'i ayarlayın veya yukarıdaki satırı düzenleyin")

In [ ]:
# RAG Pipeline konfigürasyonu
config = {
    "chunk_size": 1000,
    "chunk_overlap": 200,
    "temperature": 0.3,
    "retrieval_k": 4,
    "model_name": "gemini-1.5-flash"
}

print("Konfigürasyon:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 3. Doküman Yükleme ve İşleme

In [ ]:
# Test dokümanları için dizin kontrolü
data_dir = project_root / "data"
data_dir.mkdir(exist_ok=True)

print(f"Data dizini: {data_dir}")
print(f"Mevcut dosyalar:")

pdf_files = list(data_dir.glob("*.pdf"))
if pdf_files:
    for pdf_file in pdf_files:
        file_size = pdf_file.stat().st_size / (1024*1024)  # MB
        print(f"  📄 {pdf_file.name} ({file_size:.2f} MB)")
else:
    print("  Henüz PDF dosyası yok. Test için data/ dizinine PDF dosyaları koyabilirsiniz.")

In [ ]:
# DocumentProcessor test'i
processor = DocumentProcessor()

# Test dosya validasyonu
test_files = [
    "test.pdf",
    "document.txt",  # Geçersiz
    "large_file.pdf"  # Potansiyel olarak büyük
]

print("Dosya validasyon testleri:")
for filename in test_files:
    is_valid, error_msg = validate_file(filename, max_size_mb=10)
    status = "✅" if is_valid else "❌"
    print(f"  {status} {filename}: {error_msg or 'Geçerli'}")

In [ ]:
# RAG Pipeline oluşturma
try:
    with Timer() as timer:
        rag_pipeline = create_rag_pipeline(config)
    
    print(f"✅ RAG Pipeline oluşturuldu ({timer.elapsed():.2f}s)")
    print(f"Pipeline türü: {type(rag_pipeline).__name__}")
    
except Exception as e:
    print(f"❌ RAG Pipeline oluşturma hatası: {e}")
    rag_pipeline = None

In [ ]:
# PDF dosyalarını yükle ve işle (eğer varsa)
if pdf_files and rag_pipeline:
    try:
        with Timer() as timer:
            # Dokümanları yükle
            documents = rag_pipeline.load_documents([str(f) for f in pdf_files[:2]])  # İlk 2 dosya
            print(f"📚 {len(documents)} sayfa yüklendi")
            
            # Dokümanları işle (chunk'lara böl)
            chunks = rag_pipeline.process_documents(documents)
            print(f"✂️ {len(chunks)} parça oluşturuldu")
            
            # Vector store oluştur
            rag_pipeline.create_vectorstore(chunks)
            print("🔍 Vector store oluşturuldu")
            
            # RAG chain setup
            rag_pipeline.setup_rag_chain()
            print("🔗 RAG chain kuruldu")
        
        print(f"\n✅ Doküman işleme tamamlandı ({timer.elapsed():.2f}s)")
        
        # İlk birkaç chunk'ı göster
        print("\n📖 Örnek chunk'lar:")
        for i, chunk in enumerate(chunks[:2]):
            content = truncate_text(chunk.page_content, 150)
            print(f"  Chunk {i+1}: {content}...")
            
    except Exception as e:
        print(f"❌ Doküman işleme hatası: {e}")
else:
    print("ℹ️ PDF dosyası bulunamadı veya RAG pipeline hazır değil")
    print("Test için data/ dizinine PDF dosyaları ekleyin")

## 4. RAG Pipeline Test'i

In [ ]:
# Basit test sorguları
test_queries = [
    "Bu dokümanın ana konusu nedir?",
    "Önemli noktalar nelerdir?",
    "Hangi teknolojilerden bahsediliyor?",
    "Sonuç ve öneriler bölümünde ne yazıyor?"
]

if rag_pipeline and hasattr(rag_pipeline, 'chain') and rag_pipeline.chain:
    print("🧪 RAG Pipeline Test Sorguları\n")
    
    for i, query in enumerate(test_queries[:2], 1):  # İlk 2 soruyu test et
        print(f"Soru {i}: {query}")
        print("-" * 50)
        
        try:
            with Timer() as timer:
                response = rag_pipeline.query(query)
            
            answer = response.get("answer", "Yanıt alınamadı")
            sources = response.get("sources", [])
            
            print(f"💬 Yanıt ({timer.elapsed():.2f}s):")
            print(truncate_text(answer, 300))
            
            if sources:
                print(f"\n📚 Kaynaklar ({len(sources)} adet):")
                for j, source in enumerate(sources[:2]):
                    source_text = truncate_text(source.get('content', ''), 100)
                    print(f"  {j+1}. {source_text}...")
            
            print("\n" + "=" * 70 + "\n")
            
        except Exception as e:
            print(f"❌ Sorgu hatası: {e}\n")
else:
    print("ℹ️ RAG pipeline hazır değil. Önce dokümanları yükleyin.")

## 5. Performans Değerlendirmesi

In [ ]:
# Performans metrikleri
if rag_pipeline and hasattr(rag_pipeline, 'vectorstore') and rag_pipeline.vectorstore:
    print("📊 Sistem Performans Bilgileri\n")
    
    # Vector store bilgileri
    try:
        # Chunk sayısı
        collection = rag_pipeline.vectorstore._collection
        chunk_count = collection.count()
        print(f"📚 Toplam chunk sayısı: {chunk_count}")
        
        # Örnek similarity search
        test_query = "teknologi"
        with Timer() as timer:
            similar_docs = rag_pipeline.vectorstore.similarity_search(test_query, k=3)
        
        print(f"🔍 Similarity search süresi: {timer.elapsed():.3f}s")
        print(f"📄 Bulunan benzer doküman sayısı: {len(similar_docs)}")
        
        # Memory kullanımı (yaklaşık)
        import sys
        total_size = sum(sys.getsizeof(doc.page_content) for doc in similar_docs)
        print(f"💾 Örnek doküman boyutu: {total_size / 1024:.2f} KB")
        
    except Exception as e:
        print(f"❌ Performans bilgisi alınamadı: {e}")
else:
    print("ℹ️ Vector store hazır değil")

## 6. İnteraktif Test Alanı

In [ ]:
# İnteraktif soru-cevap fonksiyonu
def ask_question(question: str, show_sources: bool = True):
    """RAG sistemine soru sor ve yanıtı göster"""
    if not rag_pipeline or not hasattr(rag_pipeline, 'chain') or not rag_pipeline.chain:
        return "❌ RAG pipeline hazır değil. Önce dokümanları yükleyin."
    
    try:
        print(f"❓ Soru: {question}")
        print("-" * 60)
        
        with Timer() as timer:
            response = rag_pipeline.query(question)
        
        answer = response.get("answer", "Yanıt alınamadı")
        sources = response.get("sources", [])
        
        print(f"💡 Yanıt ({timer.elapsed():.2f}s):")
        print(answer)
        
        if show_sources and sources:
            print(f"\n📚 Kaynaklar ({len(sources)} adet):")
            for i, source in enumerate(sources, 1):
                source_text = truncate_text(source.get('content', ''), 200)
                print(f"\n  {i}. {source_text}...")
                
                # Metadata bilgisi varsa göster
                if 'metadata' in source and 'source' in source['metadata']:
                    source_file = Path(source['metadata']['source']).name
                    print(f"     📄 Kaynak: {source_file}")
        
        return answer
        
    except Exception as e:
        error_msg = f"❌ Hata: {str(e)}"
        print(error_msg)
        return error_msg

# Fonksiyonu test et
print("🧪 İnteraktif test fonksiyonu hazır!")
print("Kullanım: ask_question('Sorunuz buraya')")

In [ ]:
# Örnek kullanım - bu hücreyi düzenleyerek farklı sorular sorabilirsiniz
ask_question("Bu dokümanın ana konusu nedir?")

In [ ]:
# Başka bir örnek soru
ask_question("Hangi teknolojiler kullanılıyor?")

## 7. Sistem Durumu Özeti

In [ ]:
# Sistem durumu özeti
def system_status():
    """Sistem durumunu göster"""
    print("🔍 RAG Chatbot Sistem Durumu")
    print("=" * 40)
    
    # API Key kontrolü
    api_status = "✅ Hazır" if check_api_key() else "❌ Eksik"
    print(f"🔑 API Key: {api_status}")
    
    # RAG Pipeline durumu
    pipeline_status = "✅ Hazır" if rag_pipeline else "❌ Yok"
    print(f"🔗 RAG Pipeline: {pipeline_status}")
    
    # Vector Store durumu
    vectorstore_status = "❌ Yok"
    if rag_pipeline and hasattr(rag_pipeline, 'vectorstore') and rag_pipeline.vectorstore:
        try:
            count = rag_pipeline.vectorstore._collection.count()
            vectorstore_status = f"✅ Hazır ({count} chunk)"
        except:
            vectorstore_status = "⚠️ Hatalı"
    print(f"🗄️ Vector Store: {vectorstore_status}")
    
    # Chain durumu
    chain_status = "❌ Yok"
    if rag_pipeline and hasattr(rag_pipeline, 'chain') and rag_pipeline.chain:
        chain_status = "✅ Hazır"
    print(f"🔗 RAG Chain: {chain_status}")
    
    # Dosya durumu
    pdf_count = len(list(data_dir.glob("*.pdf")))
    print(f"📁 PDF Dosyaları: {pdf_count} adet")
    
    # Sistem hazırlık durumu
    ready = all([
        check_api_key(),
        rag_pipeline is not None,
        hasattr(rag_pipeline, 'chain') and rag_pipeline.chain is not None
    ])
    
    status_icon = "✅" if ready else "❌"
    status_text = "Sistem kullanıma hazır!" if ready else "Sistem kurulumu tamamlanmamış"
    
    print(f"\n{status_icon} {status_text}")
    
    if not ready:
        print("\n📋 Yapılması gerekenler:")
        if not check_api_key():
            print("  • Gemini API Key ayarlayın")
        if not rag_pipeline:
            print("  • RAG Pipeline oluşturun")
        if pdf_count == 0:
            print("  • data/ dizinine PDF dosyaları ekleyin")
        if rag_pipeline and (not hasattr(rag_pipeline, 'chain') or not rag_pipeline.chain):
            print("  • Dokümanları yükleyin ve işleyin")

# Sistem durumunu göster
system_status()

## 8. Temizlik ve Kaynak Yönetimi

In [ ]:
# Memory temizliği (isteğe bağlı)
def cleanup_resources():
    """Sistem kaynaklarını temizle"""
    global rag_pipeline
    
    if rag_pipeline:
        # Vector store'u temizle
        if hasattr(rag_pipeline, 'vectorstore') and rag_pipeline.vectorstore:
            try:
                rag_pipeline.vectorstore._collection.delete()
                print("🗑️ Vector store temizlendi")
            except:
                pass
        
        rag_pipeline = None
        print("🧹 RAG pipeline temizlendi")
    
    # Garbage collection
    import gc
    gc.collect()
    print("♻️ Memory temizliği tamamlandı")

print("🧹 Temizlik fonksiyonu hazır: cleanup_resources()")
print("⚠️ Bu fonksiyon mevcut RAG pipeline'ını siler!")

---

## 📚 Kullanım Notları

1. **API Key**: Öncelikle Gemini API key'inizi ayarlayın
2. **PDF Dosyaları**: Test için `data/` dizinine PDF dosyaları koyun
3. **Konfigürasyon**: `config` dictionary'sini ihtiyaçlarınıza göre ayarlayın
4. **Test**: `ask_question()` fonksiyonunu kullanarak interaktif testler yapın
5. **Performans**: Büyük dosyalar için chunk_size ve retrieval_k parametrelerini optimize edin

## 🔗 Faydalı Linkler

- [LangChain Dokümantasyonu](https://python.langchain.com/)
- [ChromaDB Rehberi](https://docs.trychroma.com/)
- [Gemini API Referansı](https://ai.google.dev/gemini-api/docs)
- [Proje README](../README.md)